In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


## Import IMDB dataset

In [2]:
INDEX_FROM = 3
(X_train, y_train), (X_test, y_test) = imdb.load_data(index_from=INDEX_FROM)

In [3]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}

In [4]:
# print first sentence in X_train
sentence = [id_to_word[word] for word in X_train[0]]
print(' '.join(sentence))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sho

## Pre-processing

In [12]:
from tensorflow.keras.preprocessing import sequence

In [13]:
max_review_length = max([len(x) for x in X_train])
print(f"Longest sequence is {max_review_length}")

Longest sequence is 2494


In [14]:
X_train = sequence.pad_sequences(X_train, padding='post', maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, padding='post', maxlen=max_review_length)

## LSTM model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [24]:
nb_words = max([max(x) for x in X_train]) + 1
embedding_vector_length = 64

In [26]:
model = Sequential()
model.add(Embedding(nb_words, embedding_vector_length))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          5669568   
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 5,768,513
Trainable params: 5,768,513
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-3), 
              metrics=['accuracy'])

In [34]:
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
callbacks = [es]

In [ ]:
model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), callbacks=callbacks)

Epoch 1/1000
196/196 [==============================] - 68s 341ms/step - loss: 0.6933 - accuracy: 0.4980 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/1000
196/196 [==============================] - 64s 329ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/1000
196/196 [==============================] - 65s 331ms/step - loss: 0.6932 - accuracy: 0.5003 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/1000
196/196 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4955